# Тема “Предобработка текста с помощью Python”

Осуществим предобработку данных с Твиттера, чтобы очищенные данные в дальнейшем
использовать для задачи классификации. Данный датасет содержит негативные (label = 1)
и нейтральные (label = 0) высказывания. Для работы объединим train_df и test_df.
Задания:
1. Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим
функцию:
● для того, чтобы найти все вхождения паттерна в тексте, необходимо
использовать re.findall(pattern, input_txt)
● для для замены @user на пробел, необходимо использовать re.sub()
2. Изменим регистр твитов на нижний с помощью .lower().
3. Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя
apostrophe_dict. Для этого необходимо сделать функцию: для каждого слова в
тексте проверить (for word in text.split()), если слово есть в словаре apostrophe_dict в
качестве ключа (сокращенного слова), то заменить ключ на значение (полную
версию слова).
4. Заменим сокращения на их полные формы, используя short_word_dict. Для этого
воспользуемся функцией, используемой в предыдущем пункте.
5. Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict.
Для этого воспользуемся функцией, используемой в предыдущем пункте.
6. Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'.
7. Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'.
8. Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'.
9. Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if
len(w)>1]).
10. Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый
столбец 'tweet_token'.
11. Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец
'tweet_token_filtered' без стоп-слов.
12. Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим
столбец 'tweet_stemmed' после применения стемминга.
13. Применим лемматизацию к токенам с помощью
nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после
применения лемматизации.
14. Сохраним результат предобработки в pickle-файл.

In [1]:
import pandas as pd
import re
import nltk

In [2]:
# Словари отсюда: https://www.kaggle.com/code/gauravchhabra/nlp-twitter-sentiment-analysis-project

apostrophe_dict = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

short_word_dict = {
"121": "one to one",
"a/s/l": "age, sex, location",
"adn": "any day now",
"afaik": "as far as I know",
"afk": "away from keyboard",
"aight": "alright",
"alol": "actually laughing out loud",
"b4": "before",
"b4n": "bye for now",
"bak": "back at the keyboard",
"bf": "boyfriend",
"bff": "best friends forever",
"bfn": "bye for now",
"bg": "big grin",
"bta": "but then again",
"btw": "by the way",
"cid": "crying in disgrace",
"cnp": "continued in my next post",
"cp": "chat post",
"cu": "see you",
"cul": "see you later",
"cul8r": "see you later",
"cya": "bye",
"cyo": "see you online",
"dbau": "doing business as usual",
"fud": "fear, uncertainty, and doubt",
"fwiw": "for what it's worth",
"fyi": "for your information",
"g": "grin",
"g2g": "got to go",
"ga": "go ahead",
"gal": "get a life",
"gf": "girlfriend",
"gfn": "gone for now",
"gmbo": "giggling my butt off",
"gmta": "great minds think alike",
"h8": "hate",
"hagn": "have a good night",
"hdop": "help delete online predators",
"hhis": "hanging head in shame",
"iac": "in any case",
"ianal": "I am not a lawyer",
"ic": "I see",
"idk": "I don't know",
"imao": "in my arrogant opinion",
"imnsho": "in my not so humble opinion",
"imo": "in my opinion",
"iow": "in other words",
"ipn": "I’m posting naked",
"irl": "in real life",
"jk": "just kidding",
"l8r": "later",
"ld": "later, dude",
"ldr": "long distance relationship",
"llta": "lots and lots of thunderous applause",
"lmao": "laugh my ass off",
"lmirl": "let's meet in real life",
"lol": "laugh out loud",
"ltr": "longterm relationship",
"lulab": "love you like a brother",
"lulas": "love you like a sister",
"luv": "love",
"m/f": "male or female",
"m8": "mate",
"milf": "mother I would like to fuck",
"oll": "online love",
"omg": "oh my god",
"otoh": "on the other hand",
"pir": "parent in room",
"ppl": "people",
"r": "are",
"rofl": "roll on the floor laughing",
"rpg": "role playing games",
"ru": "are you",
"shid": "slaps head in disgust",
"somy": "sick of me yet",
"sot": "short of time",
"thanx": "thanks",
"thx": "thanks",
"ttyl": "talk to you later",
"u": "you",
"ur": "you are",
"uw": "you’re welcome",
"wb": "welcome back",
"wfm": "works for me",
"wibni": "wouldn't it be nice if",
"wtf": "what the fuck",
"wtg": "way to go",
"wtgp": "want to go private",
"ym": "young man",
"gr8": "great"
}

emoticon_dict = {
":)": "happy",
":‑)": "happy",
":-]": "happy",
":-3": "happy",
":->": "happy",
"8-)": "happy",
":-}": "happy",
":o)": "happy",
":c)": "happy",
":^)": "happy",
"=]": "happy",
"=)": "happy",
"<3": "happy",
":-(": "sad",
":(": "sad",
":c": "sad",
":<": "sad",
":[": "sad",
">:[": "sad",
":{": "sad",
">:(": "sad",
":-c": "sad",
":-< ": "sad",
":-[": "sad",
":-||": "sad"
}

In [3]:
train_df = pd.read_csv('train_tweets.csv')
test_df = pd.read_csv('test_tweets.csv')
df = pd.concat([train_df, test_df])
df

id  label                                              tweet
0          1    0.0   @user when a father is dysfunctional and is s...
1          2    0.0  @user @user thanks for #lyft credit i can't us...
2          3    0.0                                bihday your majesty
3          4    0.0  #model   i love u take with u all the time in ...
4          5    0.0             factsguide: society now    #motivation
...      ...    ...                                                ...
17192  49155    NaN  thought factory: left-right polarisation! #tru...
17193  49156    NaN  feeling like a mermaid ð #hairflip #neverre...
17194  49157    NaN  #hillary #campaigned today in #ohio((omg)) &am...
17195  49158    NaN  happy, at work conference: right mindset leads...
17196  49159    NaN  my   song "so glad" free download!  #shoegaze ...

[49159 rows x 3 columns]

In [4]:
def fix_by_word(word):
    # заменим сокращения с апострофами
    if word in apostrophe_dict.keys():
        word = apostrophe_dict[word]
    
    # заменим сокращения
    if word in short_word_dict.keys():
        word = short_word_dict[word]
    
    # заменим эмотиконы
    if word in emoticon_dict.keys():
        word = emoticon_dict[word]
        
    return word  

In [5]:
def text_preprocessing(input_txt):
    
    # удаляем @user
    input_txt = re.sub(r'@[\w]*', ' ', input_txt)
    
    # замена пунктуации на пробелы
    input_txt = re.sub(r'[^\w\s]', ' ', input_txt)
    
    # замена спец. символов 
    input_txt = re.sub(r'[^a-zA-Z0-9]', ' ', input_txt)
    
    # замена чисел
    input_txt = re.sub(r'[^a-zA-Z]', ' ', input_txt)
    
    # приводим в нижний регистр
    input_txt = input_txt.lower()
    
    # Фиксим отдельные слова
    text = ''
    for word in input_txt.split(' '):
        word = fix_by_word(word)
        text += f' {word}'
    
    # Удалим слова длиной в 1 символ
    text = ' '.join([w for w in text.split() if len(w)>1])
    
        
    return text

In [6]:
%%time

# Предобработка твитов
df['tweet'] = df.apply(lambda x: text_preprocessing(x['tweet']), axis= 1)

# Поделим твиты на токены
df['tweet_token'] = df.apply(lambda x: nltk.tokenize.word_tokenize(x['tweet']), axis= 1)

# Удаляем стоп-слова из токенов
df['tweet_token_filtered'] = df.apply(lambda x: [w for w in x['tweet_token'] \
                                                 if not w in nltk.corpus.stopwords.words('english')], axis= 1)

# Применим стемминг к токенам
stemmer=nltk.stem.PorterStemmer()
df['tweet_stemmed'] = df.apply(lambda x: [stemmer.stem(w) for w in x['tweet_token_filtered']], axis= 1)

# Применим лемматизацию
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
df['tweet_lemmatized'] = df.apply(lambda x: [lemmatizer.lemmatize(w) for w in x['tweet_stemmed']], axis= 1)

df

Wall time: 3min 3s


id  label                                              tweet  \
0          1    0.0  when father is dysfunctional and is so selfish...   
1          2    0.0  thanks for lyft credit can use cause they don ...   
2          3    0.0                                bihday your majesty   
3          4    0.0  model love you take with you all the time in y...   
4          5    0.0                  factsguide society now motivation   
...      ...    ...                                                ...   
17192  49155    NaN  thought factory left right polarisation trump ...   
17193  49156    NaN  feeling like mermaid hairflip neverready forma...   
17194  49157    NaN  hillary campaigned today in ohio oh my god amp...   
17195  49158    NaN  happy at work conference right mindset leads t...   
17196  49159    NaN  my song so glad free download shoegaze newmusi...   

                                             tweet_token  \
0      [when, father, is, dysfunctional, and, is, so,...   
1      [thanks, for, lyft, credit, can, use, cause, t...   
2                                [bihday, your, majesty]   
3      [model, love, you, take, with, you, all, the, ...   
4                 [factsguide, society, now, motivation]   
...                                                  ...   
17192  [thought, factory, left, right, polarisation, ...   
17193  [feeling, like, mermaid, hairflip, neverready,...   
17194  [hillary, campaigned, today, in, ohio, oh, my,...   
17195  [happy, at, work, conference, right, mindset, ...   
17196  [my, song, so, glad, free, download, shoegaze,...   

                                    tweet_token_filtered  \
0      [father, dysfunctional, selfish, drags, kids, ...   
1      [thanks, lyft, credit, use, cause, offer, whee...   
2                                      [bihday, majesty]   
3                              [model, love, take, time]   
4                      [factsguide, society, motivation]   
...                                                  ...   
17192  [thought, factory, left, right, polarisation, ...   
17193  [feeling, like, mermaid, hairflip, neverready,...   
17194  [hillary, campaigned, today, ohio, oh, god, am...   
17195  [happy, work, conference, right, mindset, lead...   
17196  [song, glad, free, download, shoegaze, newmusi...   

                                           tweet_stemmed  \
0      [father, dysfunct, selfish, drag, kid, dysfunc...   
1      [thank, lyft, credit, use, caus, offer, wheelc...   
2                                      [bihday, majesti]   
3                              [model, love, take, time]   
4                            [factsguid, societi, motiv]   
...                                                  ...   
17192  [thought, factori, left, right, polaris, trump...   
17193  [feel, like, mermaid, hairflip, neverreadi, fo...   
17194  [hillari, campaign, today, ohio, oh, god, amp,...   
17195  [happi, work, confer, right, mindset, lead, cu...   
17196  [song, glad, free, download, shoegaz, newmus, ...   

                                        tweet_lemmatized  
0      [father, dysfunct, selfish, drag, kid, dysfunc...  
1      [thank, lyft, credit, use, caus, offer, wheelc...  
2                                      [bihday, majesti]  
3                              [model, love, take, time]  
4                            [factsguid, societi, motiv]  
...                                                  ...  
17192  [thought, factori, left, right, polaris, trump...  
17193  [feel, like, mermaid, hairflip, neverreadi, fo...  
17194  [hillari, campaign, today, ohio, oh, god, amp,...  
17195  [happi, work, confer, right, mindset, lead, cu...  
17196  [song, glad, free, download, shoegaz, newmus, ...  

[49159 rows x 7 columns]

In [7]:
# Сохраняем результат в pickle-файл
df.to_pickle('result.pkl')